In [314]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.datasets import make_s_curve
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split



In [198]:
gss=pd.read_csv("GSS_1996.csv", encoding="latin1")

In [199]:
#Clean the data
##drop cases with no answer, not asked, and don't know on the DV
badanswers=["NA","IAP","DK"]
for answer in badanswers:
    dropindex = gss[ (gss['chnang7'] == answer)].index
    gss.drop(dropindex , inplace=True)
##drop cases with null values on the DV
gss=gss[gss["chnang7"].notna()]

In [200]:
gss["chnang7"].unique()

array(['didnt', 'did'], dtype=object)

In [201]:
#Make DV a dummy variable called pray_anger
pray_anger=[]
for answer in gss["chnang7"]:
    if answer=="didnt":
        pray_anger.append(0)
    elif answer=="did":
        pray_anger.append(1)
    else:
        print("Value Error: Please recode data to omit all values that are not 'did' or 'didn't'")

In [202]:
#Reshape the DV for sklearn
DVpray_anger=np.array(pray_anger)
DVpray_anger.reshape(-1,1)

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [298]:
#Create Dataframe to Catch Missing Values
df_missing=pd.DataFrame(columns=["Variable","Row"])

In [299]:
#################
#Recoding the IVs
#################

#Making Dummies... almost.
#We cannot recode dummy variables using the pandas get_dummies command because we want to preserve the missing data (coded .a) on each of the IVs. We are doing this so that we can multiply impute them later. Instead, we will manually loop through the data and document the location of missing values in a new dataframe.

##Sex (to dummy variable)

missingcounter=0
sex=[]
for answer in gss["sex"]:
    missingcounter+=1
    if answer=="male":
        sex.append(0)
    elif answer=="female":
        sex.append(1)
    elif answer==".a":
        race.append(answer)
        print(f"missing value on sex at row {missingcounter}")
        df_missing=df_missing.append({"Variable": "sex", "Row": missingcounter}, ignore_index=True)
    else:
        print("Sex value error: {answer}.")

##Race(To dummy variable)
missingcounter=0
race=[]
for answer in gss["race"]:
    missingcounter+=1
    if answer=="white":
        race.append(0)
    elif answer=="black" or answer=="other":
        race.append(1)
    elif answer==".a":
        race.append(answer)
        print(f"missing value on race at row {missingcounter}")
        df_missing=df_missing.append({"Variable": "race", "Row": missingcounter}, ignore_index=True)
    else:
        print("Race value error: {answer}.")
##Education(To dummy variable with having degree as reference category)
missingcounter=0
education=[]
for answer in gss["degree"]:
    missingcounter +=1
    if answer =="bachelor" or answer=="graduate":
        education.append(0)
    elif answer=="lt high" or answer =="high sch" or answer=="junior c":
        education.append(1)
    elif answer==".a":
        education.append(answer)
        print(f"missing value on education at row {missingcounter}")
        df_missing=df_missing.append({"Variable": "education", "Row": missingcounter}, ignore_index=True)
    else:
        print(f"education value error: {answer}.")
missingcounter=0

missing value on education at row 165
